<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/03_Analisis_Fase_2G.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

!ls -la /content/drive/MyDrive/TFM/3_Analisis/fase2g_rendimiento/

import sys
sys.path.insert(0, '/content/drive/MyDrive/TFM/3_Analisis/fase2g_rendimiento')

from analisis_fase_2g_config import inicializar_fase2g
estado = inicializar_fase2g()

Mounted at /content/drive
total 20
-rw------- 1 root root 11407 Dec 13 12:07 analisis_fase_2g_config.py
drwx------ 2 root root  4096 Dec 13 11:54 datos
drwx------ 2 root root  4096 Dec 13 12:01 __pycache__


10:02:45 | INFO     | ============================================================
INFO:Fase2G:============================================================
10:02:45 | INFO     | INICIALIZANDO FASE 2G: RENDIMIENTO COMPUTACIONAL
INFO:Fase2G:INICIALIZANDO FASE 2G: RENDIMIENTO COMPUTACIONAL
10:02:45 | INFO     | ============================================================
INFO:Fase2G:============================================================


Drive ya esta montado


10:02:45 | INFO     | Indice maestro cargado: 20 fotos desde indice_maestro.json
INFO:Fase2G:Indice maestro cargado: 20 fotos desde indice_maestro.json
10:02:45 | INFO     | Metricas cargadas: 2360 filas desde metricas_fusionadas.csv
INFO:Fase2G:Metricas cargadas: 2360 filas desde metricas_fusionadas.csv
10:02:45 | INFO     | Inicializacion completada correctamente
INFO:Fase2G:Inicializacion completada correctamente


In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
================================================================================
FASE 2G: RENDIMIENTO COMPUTACIONAL
================================================================================

Trabajo Fin de Master - Ciencia de Datos (UOC)
Evaluacion Comparativa de Tecnicas de Segmentacion en Fotografia de Retrato

Autor: Jesus L.
Fecha: Diciembre 2025
Version: 1.0

Objetivo:
---------
Documentar trade-offs practicos entre calidad (IoU) y recursos computacionales
(tiempo de inferencia, memoria GPU).

Tareas:
-------
1. Extraer tiempos de inferencia desde metadata del indice maestro
2. Extraer pico de memoria GPU desde metadata
3. Calcular metricas de eficiencia (IoU/tiempo)
4. Generar graficos de Pareto (frontera eficiente)
5. Analisis comparativo por modelo

Outputs:
--------
fase2g_rendimiento/
├── datos/
│   ├── rendimiento_por_config.csv
│   ├── rendimiento_por_modelo.csv
│   ├── eficiencia_por_config.csv
│   └── frontera_pareto.csv
├── pareto_tiempo_iou.png/pdf
├── pareto_memoria_iou.png/pdf
├── barplot_tiempo_modelo.png/pdf
├── scatter_eficiencia.png/pdf

================================================================================
"""

'\n================================================================================\nFASE 2G: RENDIMIENTO COMPUTACIONAL\n================================================================================\n\nTrabajo Fin de Master - Ciencia de Datos (UOC)\nEvaluacion Comparativa de Tecnicas de Segmentacion en Fotografia de Retrato\n\nAutor: Jesus L.\nFecha: Diciembre 2025\nVersion: 1.0\n\nObjetivo:\n---------\nDocumentar trade-offs practicos entre calidad (IoU) y recursos computacionales\n(tiempo de inferencia, memoria GPU).\n\nTareas:\n-------\n1. Extraer tiempos de inferencia desde metadata del indice maestro\n2. Extraer pico de memoria GPU desde metadata\n3. Calcular metricas de eficiencia (IoU/tiempo)\n4. Generar graficos de Pareto (frontera eficiente)\n5. Analisis comparativo por modelo\n\nOutputs:\n--------\nfase2g_rendimiento/\n├── datos/\n│   ├── rendimiento_por_config.csv\n│   ├── rendimiento_por_modelo.csv\n│   ├── eficiencia_por_config.csv\n│   └── frontera_pareto.csv\n├── paret

In [3]:
# =============================================================================
# IMPORTS Y CONFIGURACION
# =============================================================================

import os
import sys
import json
import logging
import warnings
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Tuple, Optional, Any

import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

In [4]:
# Montar Drive
try:
    from google.colab import drive
    if not os.path.exists('/content/drive/MyDrive'):
        drive.mount('/content/drive')
        print("Google Drive montado")
    else:
        print("Drive ya montado")
except ImportError:
    print("No en Colab")

Drive ya montado


In [5]:
# Configuracion de rutas
RUTA_BASE = Path('/content/drive/MyDrive/TFM')

RUTAS = {
    'fase2b': RUTA_BASE / '3_Analisis' / 'fase2b_correlaciones',
    'fase2g': RUTA_BASE / '3_Analisis' / 'fase2g_rendimiento',
    'output_datos': RUTA_BASE / '3_Analisis' / 'fase2g_rendimiento' / 'datos',
}

ARCHIVOS = {
    'indice_maestro': RUTA_BASE / '3_Analisis' / 'fase1_integracion' / 'indice_maestro.json',
    'metricas_fusionadas': RUTAS['fase2b'] / 'metricas_fusionadas.csv',
}

# Crear directorios
RUTAS['fase2g'].mkdir(parents=True, exist_ok=True)
RUTAS['output_datos'].mkdir(parents=True, exist_ok=True)

# Configuracion de estilos
STYLE_CONFIG = {
    'figure.dpi': 100,
    'figure.facecolor': 'white',
    'font.family': 'serif',
    'font.size': 10,
    'axes.titlesize': 12,
    'axes.labelsize': 10,
    'axes.spines.top': False,
    'axes.spines.right': False,
}

FIGSIZE = {
    'scatter': (10, 8),
    'barplot': (12, 6),
    'pareto': (11, 8),
    'panel': (14, 10),
}

PALETA_MODELOS = {
    'YOLOv8': '#2ecc71',
    'OneFormer': '#3498db',
    'Mask2Former': '#9b59b6',
    'SAM2': '#e74c3c',
    'BodyPix': '#1abc9c',
}

MAPEO_CONFIG_MODELO = {
    'yolov8': 'YOLOv8',
    'oneformer': 'OneFormer',
    'mask2former': 'Mask2Former',
    'sam2': 'SAM2',
    'bodypix': 'BodyPix',
}

# Logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s | %(levelname)-8s | %(message)s', datefmt='%H:%M:%S')
logger = logging.getLogger('Fase2G')

In [6]:
# =============================================================================
# FUNCIONES AUXILIARES
# =============================================================================

def obtener_modelo_base(codigo_config: str) -> str:
    """Extrae el nombre del modelo base desde el codigo de configuracion."""
    codigo_lower = codigo_config.lower()
    for prefijo, modelo in MAPEO_CONFIG_MODELO.items():
        if codigo_lower.startswith(prefijo):
            return modelo
    return 'Desconocido'

def cargar_indice_maestro() -> Optional[Dict]:
    """Carga el indice maestro del proyecto."""
    ruta = ARCHIVOS['indice_maestro']
    if ruta.exists():
        with open(ruta, 'r', encoding='utf-8') as f:
            indice = json.load(f)
        n_fotos = len([k for k in indice.keys() if not k.startswith('_metadata')])
        logger.info(f"Indice maestro cargado: {n_fotos} fotos")
        return indice
    logger.error(f"Indice no encontrado: {ruta}")
    return None

def cargar_metricas() -> Optional[pd.DataFrame]:
    """Carga el CSV de metricas."""
    ruta = ARCHIVOS['metricas_fusionadas']
    if ruta.exists():
        df = pd.read_csv(ruta)
        logger.info(f"Metricas cargadas: {len(df)} filas")
        return df
    logger.error(f"Metricas no encontradas: {ruta}")
    return None

def guardar_figura(fig: plt.Figure, nombre: str) -> None:
    """Guarda figura en PNG y PDF."""
    for fmt in ['png', 'pdf']:
        ruta = RUTAS['fase2g'] / f'{nombre}.{fmt}'
        fig.savefig(ruta, dpi=300, bbox_inches='tight', facecolor='white')
    logger.info(f"  Guardado: {nombre}")

In [7]:
# =============================================================================
# EXTRACCION DE DATOS
# =============================================================================

def extraer_rendimiento_indice(indice: Dict) -> pd.DataFrame:
    """Extrae datos de rendimiento desde el indice maestro."""
    registros = []
    fotos = [k for k in indice.keys() if not k.startswith('_metadata')]

    for foto_id in fotos:
        info_foto = indice[foto_id]
        if 'modelos_disponibles' not in info_foto:
            continue

        for config_codigo, config_data in info_foto['modelos_disponibles'].items():
            resumen = config_data.get('resumen_metadata', {})
            tiempo_ms = resumen.get('tiempo_inferencia_ms')
            memoria_mb = resumen.get('gpu_pico_mb')

            # Solo registrar si hay al menos tiempo de inferencia
            if tiempo_ms is not None:
                registros.append({
                    'foto_id': foto_id,
                    'codigo_config': config_codigo,
                    'modelo_base': obtener_modelo_base(config_codigo),
                    'tiempo_ms': float(tiempo_ms),
                    'memoria_mb': float(memoria_mb) if memoria_mb is not None else np.nan,
                })

    df = pd.DataFrame(registros)

    if len(df) > 0:
        modelos_con_datos = df['modelo_base'].unique()
        logger.info(f"Extraidos {len(df)} registros de rendimiento")
        logger.info(f"Modelos con datos: {list(modelos_con_datos)}")
    else:
        logger.warning("No se encontraron datos de rendimiento")

    return df

def fusionar_con_metricas(df_rendimiento: pd.DataFrame, df_metricas: pd.DataFrame) -> pd.DataFrame:
    """Fusiona rendimiento con metricas de IoU."""

    # Identificar columna de foto
    if 'codigo_foto' in df_metricas.columns:
        col_foto = 'codigo_foto'
    elif 'foto_id' in df_metricas.columns:
        col_foto = 'foto_id'
    else:
        logger.error("No se encontro columna de foto")
        return pd.DataFrame()

    # Identificar columna de config
    if 'codigo_config' in df_metricas.columns:
        col_config = 'codigo_config'
    elif 'config_codigo' in df_metricas.columns:
        col_config = 'config_codigo'
    else:
        logger.error("No se encontro columna de config")
        return pd.DataFrame()

    logger.info(f"Columnas: foto='{col_foto}', config='{col_config}'")

    # Seleccionar columnas
    cols = [col_foto, col_config]
    for c in ['iou', 'dice', 'precision', 'recall']:
        if c in df_metricas.columns:
            cols.append(c)

    df_met = df_metricas[cols].copy()
    df_met = df_met.rename(columns={col_foto: 'foto_id', col_config: 'codigo_config'})

    # Merge
    df_fusionado = pd.merge(df_rendimiento, df_met, on=['foto_id', 'codigo_config'], how='inner')
    logger.info(f"Fusion: {len(df_fusionado)} registros")

    return df_fusionado

def calcular_estadisticas(df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Calcula estadisticas por config y por modelo."""

    # Por configuracion
    stats_config = df.groupby('codigo_config').agg({
        'modelo_base': 'first',
        'tiempo_ms': ['mean', 'std'],
        'memoria_mb': ['mean', 'std'],
        'iou': ['mean', 'std'],
    }).round(4)
    stats_config.columns = ['_'.join(col) for col in stats_config.columns]
    stats_config = stats_config.reset_index()
    stats_config = stats_config.rename(columns={
        'modelo_base_first': 'modelo_base',
        'tiempo_ms_mean': 'tiempo_ms_medio',
        'memoria_mb_mean': 'memoria_mb_medio',
        'iou_mean': 'iou_medio',
    })
    stats_config['eficiencia'] = (stats_config['iou_medio'] / stats_config['tiempo_ms_medio'] * 1000).round(4)

    # Por modelo
    stats_modelo = df.groupby('modelo_base').agg({
        'tiempo_ms': ['mean', 'std', 'count'],
        'memoria_mb': ['mean', 'std'],
        'iou': ['mean', 'std'],
    }).round(4)
    stats_modelo.columns = ['_'.join(col) for col in stats_modelo.columns]
    stats_modelo = stats_modelo.reset_index()
    stats_modelo = stats_modelo.rename(columns={
        'tiempo_ms_mean': 'tiempo_ms_medio',
        'tiempo_ms_count': 'n_registros',
        'memoria_mb_mean': 'memoria_mb_medio',
        'iou_mean': 'iou_medio',
    })
    stats_modelo['eficiencia'] = (stats_modelo['iou_medio'] / stats_modelo['tiempo_ms_medio'] * 1000).round(4)

    return stats_config, stats_modelo

def identificar_pareto(df: pd.DataFrame, col_x: str, col_y: str) -> pd.DataFrame:
    """Identifica frontera de Pareto (minimizar X, maximizar Y)."""
    df_sorted = df.sort_values(col_x).copy()
    pareto = []
    mejor_y = -np.inf

    for _, row in df_sorted.iterrows():
        if row[col_y] > mejor_y:
            pareto.append(row)
            mejor_y = row[col_y]

    df_pareto = pd.DataFrame(pareto)
    logger.info(f"Frontera Pareto: {len(df_pareto)} de {len(df)}")
    return df_pareto

In [8]:
# =============================================================================
# VISUALIZACIONES
# =============================================================================

def plot_pareto_tiempo_iou(stats_config: pd.DataFrame, pareto: pd.DataFrame) -> plt.Figure:
    """Grafico Pareto: Tiempo vs IoU."""
    fig, ax = plt.subplots(figsize=FIGSIZE['pareto'])

    for modelo, color in PALETA_MODELOS.items():
        mask = stats_config['modelo_base'] == modelo
        if mask.sum() > 0:
            ax.scatter(stats_config.loc[mask, 'tiempo_ms_medio'],
                      stats_config.loc[mask, 'iou_medio'],
                      c=color, label=modelo, alpha=0.6, s=60, edgecolors='white')

    if len(pareto) > 1:
        pareto_sorted = pareto.sort_values('tiempo_ms_medio')
        ax.plot(pareto_sorted['tiempo_ms_medio'], pareto_sorted['iou_medio'],
               'k--', linewidth=2, alpha=0.7, label='Frontera Pareto')
        ax.scatter(pareto_sorted['tiempo_ms_medio'], pareto_sorted['iou_medio'],
                  c='gold', s=150, marker='*', edgecolors='black', zorder=5)

    ax.set_xlabel('Tiempo de Inferencia (ms)')
    ax.set_ylabel('IoU Medio')
    ax.set_title('Trade-off Tiempo vs Calidad: Frontera de Pareto')
    ax.legend(loc='lower right')
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    return fig

def plot_pareto_memoria_iou(stats_config: pd.DataFrame, pareto: pd.DataFrame) -> plt.Figure:
    """Grafico Pareto: Memoria vs IoU."""
    fig, ax = plt.subplots(figsize=FIGSIZE['pareto'])

    for modelo, color in PALETA_MODELOS.items():
        mask = stats_config['modelo_base'] == modelo
        if mask.sum() > 0:
            ax.scatter(stats_config.loc[mask, 'memoria_mb_medio'],
                      stats_config.loc[mask, 'iou_medio'],
                      c=color, label=modelo, alpha=0.6, s=60, edgecolors='white')

    if len(pareto) > 1:
        pareto_sorted = pareto.sort_values('memoria_mb_medio')
        ax.plot(pareto_sorted['memoria_mb_medio'], pareto_sorted['iou_medio'],
               'k--', linewidth=2, alpha=0.7, label='Frontera Pareto')
        ax.scatter(pareto_sorted['memoria_mb_medio'], pareto_sorted['iou_medio'],
                  c='gold', s=150, marker='*', edgecolors='black', zorder=5)

    ax.set_xlabel('Memoria GPU Pico (MB)')
    ax.set_ylabel('IoU Medio')
    ax.set_title('Trade-off Memoria vs Calidad: Frontera de Pareto')
    ax.legend(loc='lower right')
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    return fig

def plot_barras_modelo(stats_modelo: pd.DataFrame) -> plt.Figure:
    """Barplot de tiempo y memoria por modelo."""
    fig, axes = plt.subplots(1, 2, figsize=FIGSIZE['panel'])

    stats_sorted = stats_modelo.sort_values('tiempo_ms_medio')
    colores = [PALETA_MODELOS.get(m, '#333') for m in stats_sorted['modelo_base']]

    # Tiempo
    axes[0].barh(stats_sorted['modelo_base'], stats_sorted['tiempo_ms_medio'], color=colores, alpha=0.8)
    axes[0].set_xlabel('Tiempo de Inferencia (ms)')
    axes[0].set_title('Tiempo Medio por Modelo')

    # Memoria
    stats_mem = stats_modelo.sort_values('memoria_mb_medio')
    colores_mem = [PALETA_MODELOS.get(m, '#333') for m in stats_mem['modelo_base']]
    axes[1].barh(stats_mem['modelo_base'], stats_mem['memoria_mb_medio'], color=colores_mem, alpha=0.8)
    axes[1].set_xlabel('Memoria GPU Pico (MB)')
    axes[1].set_title('Memoria Media por Modelo')

    fig.suptitle('Recursos Computacionales por Modelo', fontsize=14, fontweight='bold')
    plt.tight_layout()
    return fig

def plot_eficiencia(stats_config: pd.DataFrame) -> plt.Figure:
    """Scatter de eficiencia vs IoU."""
    fig, ax = plt.subplots(figsize=FIGSIZE['scatter'])

    for modelo, color in PALETA_MODELOS.items():
        mask = stats_config['modelo_base'] == modelo
        if mask.sum() > 0:
            ax.scatter(stats_config.loc[mask, 'iou_medio'],
                      stats_config.loc[mask, 'eficiencia'],
                      c=color, label=modelo, alpha=0.7, s=80, edgecolors='white')

    ax.set_xlabel('IoU Medio')
    ax.set_ylabel('Eficiencia (IoU x 1000 / ms)')
    ax.set_title('Eficiencia Computacional vs Calidad')
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    return fig

In [9]:
# =============================================================================
# EJECUCION PRINCIPAL
# =============================================================================

def ejecutar_fase2g():
    """Ejecuta Fase 2G completa."""
    logger.info("="*60)
    logger.info("FASE 2G: RENDIMIENTO COMPUTACIONAL")
    logger.info("="*60)

    # Cargar datos
    indice = cargar_indice_maestro()
    df_metricas = cargar_metricas()

    if indice is None or df_metricas is None:
        return {'exito': False, 'error': 'Error cargando datos'}

    # Extraer rendimiento
    df_rendimiento = extraer_rendimiento_indice(indice)

    if len(df_rendimiento) == 0:
        return {'exito': False, 'error': 'No hay datos de rendimiento'}

    # Fusionar
    df_fusionado = fusionar_con_metricas(df_rendimiento, df_metricas)

    if len(df_fusionado) == 0:
        return {'exito': False, 'error': 'Fusion vacia'}

    # Estadisticas
    stats_config, stats_modelo = calcular_estadisticas(df_fusionado)

    # Pareto
    pareto_tiempo = identificar_pareto(stats_config, 'tiempo_ms_medio', 'iou_medio')

    df_con_memoria = stats_config.dropna(subset=['memoria_mb_medio'])
    if len(df_con_memoria) > 0:
        pareto_memoria = identificar_pareto(df_con_memoria, 'memoria_mb_medio', 'iou_medio')
    else:
        pareto_memoria = pd.DataFrame()

    # Visualizaciones
    logger.info("-"*50)
    logger.info("GENERANDO VISUALIZACIONES")
    logger.info("-"*50)

    plt.style.use('seaborn-v0_8-whitegrid')
    plt.rcParams.update(STYLE_CONFIG)

    fig = plot_pareto_tiempo_iou(stats_config, pareto_tiempo)
    guardar_figura(fig, 'pareto_tiempo_iou')
    plt.close(fig)

    if len(pareto_memoria) > 0:
        fig = plot_pareto_memoria_iou(stats_config, pareto_memoria)
        guardar_figura(fig, 'pareto_memoria_iou')
        plt.close(fig)

    fig = plot_barras_modelo(stats_modelo)
    guardar_figura(fig, 'barplot_tiempo_modelo')
    plt.close(fig)

    fig = plot_eficiencia(stats_config)
    guardar_figura(fig, 'scatter_eficiencia')
    plt.close(fig)

    # Exportar CSVs
    logger.info("-"*50)
    logger.info("EXPORTANDO DATOS")
    logger.info("-"*50)

    stats_config.to_csv(RUTAS['output_datos'] / 'rendimiento_por_config.csv', index=False)
    stats_modelo.to_csv(RUTAS['output_datos'] / 'rendimiento_por_modelo.csv', index=False)
    pareto_tiempo.to_csv(RUTAS['output_datos'] / 'frontera_pareto_tiempo.csv', index=False)
    logger.info("  CSVs guardados")


    logger.info("="*60)
    logger.info("FASE 2G COMPLETADA")
    logger.info(f"Resultados en: {RUTAS['fase2g']}")
    logger.info("="*60)

    return {
        'exito': True,
        'registros': len(df_fusionado),
        'modelos': len(stats_modelo),
        'pareto': len(pareto_tiempo),
    }

In [10]:
# =============================================================================
# MAIN
# =============================================================================

if __name__ == '__main__':
    resultados = ejecutar_fase2g()
    print(f"\nResultados: {resultados}")

10:02:46 | INFO     | ============================================================
INFO:Fase2G:============================================================
10:02:46 | INFO     | FASE 2G: RENDIMIENTO COMPUTACIONAL
INFO:Fase2G:FASE 2G: RENDIMIENTO COMPUTACIONAL
10:02:46 | INFO     | ============================================================
INFO:Fase2G:============================================================
10:02:46 | INFO     | Indice maestro cargado: 20 fotos
INFO:Fase2G:Indice maestro cargado: 20 fotos
10:02:46 | INFO     | Metricas cargadas: 2360 filas
INFO:Fase2G:Metricas cargadas: 2360 filas
10:02:46 | INFO     | Extraidos 1660 registros de rendimiento
INFO:Fase2G:Extraidos 1660 registros de rendimiento
10:02:46 | INFO     | Modelos con datos: ['Mask2Former', 'SAM2', 'YOLOv8']
INFO:Fase2G:Modelos con datos: ['Mask2Former', 'SAM2', 'YOLOv8']
10:02:46 | INFO     | Columnas: foto='codigo_foto', config='config_codigo'
INFO:Fase2G:Columnas: foto='codigo_foto', config='config_codi


Resultados: {'exito': True, 'registros': 1363, 'modelos': 3, 'pareto': 4}
